Модуль для нормализации изображения найденного автомобильного номера. Нейросеть обучена на изображениях формата .jpg размером 1280\*720 и 1920\*1080. Маски подготовлены с помощью программы LabelImg.

In [ ]:
# коллабные импорты
from google.colab import drive
from google.colab import files


drive.mount('/GD',force_remount=True)
!cp -r /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/data.zip .
!cp -r /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/masks_jpg.zip .
!cp -r /GD/'My Drive'/'Colab Notebooks'/PlateRecognition/masks_jpg_predict.zip .

!unzip data.zip
!unzip masks_jpg.zip
!unzip masks_jpg_predict.zip

In [2]:
!ls

data.zip   masks_jpg_predict	  masks_jpg.zip  pics_jpg
masks_jpg  masks_jpg_predict.zip  masks_xml	 sample_data


In [3]:
import math
import sys
import os
import glob
import shutil
import cv2
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd

from random import choice, sample
from PIL import Image
from tqdm.notebook import tqdm

Для начала разобьем найденные маски на фото на отдельные изображения